In [ ]:
import os
from dotenv import load_dotenv
# load_dotenv("/home/liuyiwen/AI/AI Agent/quantum_news_agent/project/.env")
# GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_API_KEY'] = ''

In [2]:
import importlib
mod = importlib.import_module("google.adk.tools.mcp_tool.mcp_session_manager")
print([n for n in dir(mod) if "Params" in n or "Server" in n])


['SseServerParams', 'StdioServerParameters', 'StreamableHTTPServerParams']


In [9]:
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StreamableHTTPServerParams
from google.adk.agents.llm_agent import LlmAgent

# 1. Configure MCP server connection
MCP_URL = "https://elene-unrequired-dario.ngrok-free.dev/mcp"
#MCP_URL = "http://localhost:8000"

tools = MCPToolset(
    connection_params=StreamableHTTPServerParams(url=MCP_URL)
)



# 2. Build agent
PROMPT = "you have access to MCP tools (send_email tool and read_unread_email tool) and also MCP prompt, use them to assist the user "

agent =LlmAgent(
    model="gemini-2.0-flash",
    name="news_assistant",
    description="Agent integrated with MCP HTTP server.",
    instruction=PROMPT,
    tools=[tools],
    
)

# 3. Session + runner
session_service = InMemorySessionService()
session = await session_service.create_session(
    state={}, app_name="quantume_news_app", user_id="user_fs"
)



runner = Runner(app_name="quantume_news_app", agent=agent, session_service=session_service)

# 4. Run agent async
query = "write sweet email to tell a girl she is super beautiful and send the email to liuyiwen153@gmail.com"
#query = "check the latest unread email and reply with polite messages to the sender" 
content = types.Content(role="user", parts=[types.Part(text=query)])

async for event in runner.run_async(session_id=session.id, user_id=session.user_id, new_message=content):
    if event.is_final_response():
        if event.content and event.content.parts:
            print(event.content.parts[0].text)
        elif getattr(event, "actions", None) and getattr(event.actions, "escalate", False):
            print(f"Agent escalated: {event.error_message or 'Unknown escalation.'}")
        break



McpError: Session terminated